#### ID 2061

```Count the number of users who made more than 5 searches in August 2021.```

In [ ]:
%%sql
SELECT COUNT(user_id)
FROM (SELECT user_id, COUNT(search_id) AS cnt
      FROM fb_searches
      WHERE EXTRACT(MONTH FROM date) = 8
        AND EXTRACT(YEAR FROM date) = 2021
      GROUP BY user_id
      HAVING COUNT(search_id) > 5) AS sq

In [ ]:
df = fb_searches
df.query('date.dt.year == 2021 & date.dt.month == 8').groupby('user_id', as_index=False).agg(cnt=('search_id', 'count')).query('cnt > 5')['user_id'].count()